In [1]:
#Data used is the Electrochemical Impedance Spectroscopy(EIS) test impedance Values. Capacity is predicted
#Predictions are made on new EIS test results 8 sets

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

                  #handling data
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
                 #scaling and splitting data
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
                #build and train model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import metrics
                #save and load the model
from tensorflow.keras.models import load_model
                #evaluate model
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score \

from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, GlobalMaxPooling2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.models import Model

import sys
sys.setrecursionlimit(10000)    #used to increase the recursion limit.



In [ ]:
# Data from adjacent columns is the data corresponding to 1 EIS test. (0,1);(1,2);(2,3)
# Total columns = 414 which implies 207 sets of data
# One column represents values on real axis and other on imaginary axis
# Every cycle, EIS test was done once after charging the cell to 100% SoC adn discharging it. 

In [3]:
data = pd.read_excel("C:\\Users\\RECL\\Documents\\PRADYUMNA\\Experiment Data\\CNN\\EIS Impedance Curves\\Battery Aging EIS Test.xlsx", sheet_name='Original Data', header=None)
data = data.drop(data.index[[0,1]])
data = data.drop(data.columns[[414,415]], axis=1)
data

,0,1,2,3,4,5,6,7,8,9,...,404,405,406,407,408,409,410,411,412,413
2,0.02879,0.00226,0.02879,0.00226,0.02897,0.00274,0.02924,0.00225,0.02873,0.00231,...,0.03581,0.00202,0.03576,0.00209,0.03576,-0.0021,-0.03581,-0.00206,-0.03583,-0.00209
3,0.02903,0.00127,0.02903,0.00127,0.02924,0.00123,0.02947,0.00129,0.02894,0.00133,...,0.03605,0.00111,0.03599,0.00118,0.03598,-0.00119,-0.03607,-0.000201,-0.03606,-0.00118
4,0.02934,0.000457,0.02934,0.000457,0.02952,0.00045,0.02975,0.000512,0.02924,0.000546,...,0.03633,0.000355,0.03626,0.000432,0.03625,-0.00046,-0.03632,-0.000413,-0.03632,-0.000437
5,0.02968,-0.000178,0.02968,-0.000178,0.02987,-0.00015,0.03009,-0.000092,0.02956,-0.000074,...,0.03665,-0.000223,0.03655,-0.000142,0.03655,0.000117,-0.03663,0.000168,-0.03663,0.000143
6,0.03005,-0.000671,0.03005,-0.000671,0.03024,-0.000623,0.03044,-0.000559,0.02991,-0.000551,...,0.03698,-0.000686,0.03687,-0.000595,0.03686,0.000569,-0.03695,0.000626,-0.03694,0.000595
7,0.03045,-0.00104,0.03045,-0.00104,0.03061,-0.000964,0.03079,-0.000903,0.03027,-0.000911,...,0.03732,-0.00106,0.0372,-0.000952,0.03719,0.000928,-0.03729,0.000987,-0.03728,0.000955
8,0.03085,-0.00129,0.03085,-0.00129,0.03098,-0.00122,0.03115,-0.00115,0.03063,-0.00117,...,0.03766,-0.00153,0.03754,-0.00122,0.03751,0.0012,-0.03762,0.00125,-0.03761,0.00122
9,0.03123,-0.00146,0.03123,-0.00146,0.03136,-0.00138,0.03152,-0.0013,0.03101,-0.00133,...,0.03803,-0.00154,0.03787,-0.00143,0.03785,0.00116,-0.03796,0.00146,-0.03794,0.00143
10,0.03161,-0.00156,0.03161,-0.00156,0.0317,-0.00147,0.03186,-0.0014,0.03136,-0.00144,...,0.03836,-0.00169,0.03817,-0.00159,0.03817,0.00157,-0.0383,0.00163,-0.03827,0.00159
11,0.03197,-0.00159,0.03197,-0.00159,0.03204,-0.0015,0.0322,-0.00143,0.03168,-0.00148,...,0.03869,-0.00183,0.0385,-0.0017,0.03847,0.00169,-0.03862,0.00174,-0.03859,0.00171


In [4]:
outdata = pd.read_excel("C:\\Users\\RECL\\Documents\\PRADYUMNA\\Experiment Data\\CNN\\EIS Impedance Curves\\Capacity Output.xlsx", sheet_name= 'Sheet1', header=None)
outdata = outdata.drop(outdata.index[0])
outdata = outdata.drop(outdata.columns[1],axis=1)
outdata

,0
1,99.287719
2,98.778947
3,98.498246
4,98.150877
5,97.915789
...,...
203,75.742181
204,75.373637
205,75.602828
206,76.032622


In [5]:
# For Training Data 
z = {}

for k in range(0,207):    
    z[k] = data[[(2*k), ((2*k)+1)]]

for k in z:
    z[k]= z[k].to_numpy()
    z[k] = np.expand_dims((z[k]), -1)

In [6]:
prediction_data = pd.read_excel("C:\\Users\\RECL\\Documents\\PRADYUMNA\\Experiment Data\\CNN\\EIS Impedance Curves\\Prediction Data.xlsx", sheet_name='Original',header=None)
prediction_data = prediction_data.drop(prediction_data.index[[0,1]])
prediction_data = prediction_data.reset_index(drop=True)
prediction_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.020976,0.001413,0.02059,0.000935,0.020537,0.00233,0.020637,0.001689,0.020686,0.001848,0.020942,0.001645,0.020703,0.001468,0.020829,0.001372
1,0.021379,0.000319,0.021108,0.000022,0.020985,0.001021,0.020979,0.00053,0.021061,0.000571,0.021376,0.000486,0.020931,0.000297,0.021294,0.000242
2,0.021682,-0.000616,0.021584,-0.000867,0.021698,0.00003,0.021572,-0.000303,0.021597,-0.000257,0.021869,-0.000466,0.021244,-0.0005,0.02183,-0.000609
3,0.022125,-0.001334,0.022197,-0.001519,0.021929,-0.000817,0.021935,-0.001001,0.022219,-0.000816,0.022414,-0.001238,0.021973,-0.001171,0.022266,-0.001312
4,0.022639,-0.001839,0.022785,-0.002005,0.022554,-0.001407,0.022507,-0.001727,0.022711,-0.001651,0.023005,-0.001861,0.022402,-0.001763,0.022914,-0.001884
5,0.023246,-0.002242,0.023401,-0.002351,0.023129,-0.001919,0.023165,-0.002139,0.023311,-0.001988,0.023632,-0.002339,0.023042,-0.002323,0.023503,-0.002368
6,0.023913,-0.002583,0.024001,-0.002656,0.023782,-0.00235,0.02382,-0.002572,0.023923,-0.002524,0.024281,-0.002705,0.023783,-0.002615,0.024089,-0.002672
7,0.024592,-0.002838,0.024572,-0.002914,0.02445,-0.002701,0.02446,-0.00283,0.024491,-0.002811,0.024942,-0.002963,0.024342,-0.002791,0.024739,-0.002893
8,0.025238,-0.00301,0.025214,-0.003014,0.025103,-0.002874,0.025085,-0.003047,0.025184,-0.002933,0.025599,-0.003128,0.024935,-0.002994,0.025397,-0.003052
9,0.025884,-0.003091,0.025836,-0.003068,0.02576,-0.002954,0.025677,-0.003108,0.025864,-0.003016,0.026246,-0.003202,0.02557,-0.003115,0.026033,-0.003139


In [7]:
y = {}   #for prediction with new EIS test data

for x in range(0,8):
    y[x] = prediction_data[[(2*x), ((2*x)+1)]]


for x in y:
    y[x]= y[x].to_numpy()
    y[x] = np.expand_dims((y[x]), -1)

In [8]:
input = np.empty((len(z.keys()), *z[0].shape))

for i, k in enumerate(z.keys()):
    input[i] = z[k]
    
pred_in = np.empty((len(y.keys()), *y[0].shape))

for i, k in enumerate(y.keys()):
    pred_in[i] = y[k]

print(input.shape) 

(207, 41, 2, 1)


In [9]:
"len() method :", len(z)

('len() method :', 207)

In [10]:
np.shape(input)

(207, 41, 2, 1)

In [11]:
train_in, val_in, train_out, val_out = train_test_split(input, outdata, test_size=0.2, random_state=100)

In [12]:
train_in

array([[[[ 0.03484   ],
         [ 0.00223   ]],

        [[ 0.03505   ],
         [ 0.00133   ]],

        [[ 0.0353    ],
         [ 0.00058315]],

        ...,

        [[ 0.07511   ],
         [-0.01845   ]],

        [[ 0.07762   ],
         [-0.01816   ]],

        [[ 0.08001   ],
         [-0.01802   ]]],


       [[[ 0.03422   ],
         [ 0.00244   ]],

        [[ 0.0344    ],
         [ 0.0015    ]],

        [[ 0.03462   ],
         [ 0.00073362]],

        ...,

        [[ 0.05859   ],
         [-0.00822   ]],

        [[ 0.05952   ],
         [-0.0085    ]],

        [[ 0.06046   ],
         [-0.00892   ]]],


       [[[ 0.03163   ],
         [ 0.00233   ]],

        [[ 0.03185   ],
         [ 0.00138   ]],

        [[ 0.03212   ],
         [ 0.00060371]],

        ...,

        [[ 0.0425    ],
         [-0.00365   ]],

        [[ 0.04288   ],
         [-0.00409   ]],

        [[ 0.0433    ],
         [-0.00462   ]]],


       ...,


       [[[ 0.03386   ],
         [ 0.0

In [13]:
train_in.shape

(165, 41, 2, 1)

In [14]:
val_out

,0
163,79.484459
189,76.299458
60,89.295564
82,87.713029
190,76.515673
65,88.997762
162,79.081219
125,83.792259
175,77.777586
137,82.68085


In [15]:
model = load_model('SoHImpedance2by2filter.h5')

In [ ]:
p = Input(shape=(41,2,1))

x = Conv2D(16,(3,3), strides=1, activation='relu', data_format='channels_last', padding='same')(p)   
x = BatchNormalization()(x)

x = Conv2D(32,(3,3), activation='relu', data_format='channels_last' , padding='same')(x)  
x = BatchNormalization()(x)

x = Flatten()(x)

x = Dense(320, activation='relu')(x)
x = Dense(160, activation='relu')(x)
x = Dense(80, activation='relu')(x)
x = Dense(1)(x)

model = Model(p,x)

model.compile(optimizer='adam',loss='mse', metrics=['mae', 'mape'])

#early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

model.fit(x=train_in, y=train_out,validation_data=(val_in,val_out),steps_per_epoch=None, epochs=220)
#, callbacks=[early_stop])

loss = pd.DataFrame(model.history.history)
loss.columns = ['Loss', 'MAE', 'MAPE%','Val_Loss', 'Val_MAE', 'Val_MAPE%']
loss

loss.plot()           #to plot the loss values
plt.xlabel('Epoch')
plt.ylabel('Loss')

keras.utils.plot_model(model, "123.png", show_shapes=True)

model.summary()

In [18]:
pred_out = pd.read_excel("C:\\Users\\RECL\\Documents\\PRADYUMNA\\Experiment Data\\CNN\\EIS Impedance Curves\\Prediction Data.xlsx", sheet_name='Capacity',header=None)
pred_out = pred_out.drop(pred_out.index[0])
pred_out = pred_out.reset_index(drop=True)
pred_out

,0
0,98.167368
1,97.90864
2,98.185388
3,98.295586
4,98.210678
5,98.06892
6,98.376434
7,98.370302


In [19]:
predictions = model.predict(pred_in)

True_value = pd.DataFrame(pred_out)
True_value = True_value.reset_index(drop=True)
predected_value = pd.DataFrame(predictions)
Error = predected_value - True_value
Error = Error.abs()  # converts -ve numbers into +ve

compare = pd.concat([True_value,predected_value,Error],axis=1)
compare.columns = ['True_value','predected_value','Error']
Error.columns=['Error']


compare

,True_value,predected_value,Error
0,98.167368,98.323631,0.156263
1,97.90864,98.480202,0.571562
2,98.185388,98.159691,0.025697
3,98.295586,98.320396,0.024811
4,98.210678,98.255898,0.04522
5,98.06892,98.355019,0.286098
6,98.376434,98.379265,0.00283
7,98.370302,98.398018,0.027716


In [20]:
Error.max()  

Error    0.571562
dtype: object

In [21]:
mean_absolute_error(pred_out,predictions)

0.14252463912988667

In [22]:
np.sqrt(mean_squared_error(pred_out,predictions)) #RMS error

0.23373342022144586

In [ ]:
#model.save('SoHImpedance2by2filter.h5')

In [ ]:
#loss.to_excel("metrics2by2.xlsx")

In [ ]:
#compare.to_excel("predictions2by2.xlsx")